In [76]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [80]:
x = np.loadtxt("x_L30.txt").reshape(
    -1, 30, 30
)  # Assuming the data is already in this shape
y = np.loadtxt("y_L30.txt")

# Convert to tensors
x_tensor = torch.tensor(x, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Split the data
x_train, x_temp, y_train, y_temp = train_test_split(
    x_tensor, y_tensor, test_size=0.3, random_state=42
)
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=(1 / 3), random_state=42
)

# Create data loaders
train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val, y_val), batch_size=64, shuffle=False)

# Check the shapes
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)
x_train = x_train.float()
x_val = x_val.float()
x_test = x_test.float()
# Create data loaders
train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val, y_val), batch_size=64, shuffle=False)
test_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=64, shuffle=False)

torch.Size([7000, 30, 30]) torch.Size([7000])
torch.Size([2000, 30, 30]) torch.Size([2000])
torch.Size([1000, 30, 30]) torch.Size([1000])


In [82]:
class IsingModelNN(nn.Module):
    def __init__(self):
        super(IsingModelNN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(30 * 30, 200), nn.ReLU(), nn.Linear(200, 2)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = IsingModelNN()

In [83]:
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()


def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for X, y in dataloader:
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()


def evaluate(dataloader, model, loss_fn):
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            total_loss += loss_fn(pred, y).item()
            total_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    num_samples = len(dataloader.dataset)
    avg_accuracy = 100 * total_correct / num_samples
    avg_loss = total_loss / len(dataloader)
    return avg_accuracy, avg_loss


epochs = 50
best_val_acc = 0
best_model = None

for epoch in range(epochs):
    train(train_loader, model, criterion, optimizer)
    train_acc, train_loss = evaluate(train_loader, model, criterion)
    val_acc, val_loss = evaluate(val_loader, model, criterion)
    print(f"Epoch {epoch+1}, Train Acc: {train_acc:.1f}%, Val Acc: {val_acc:.1f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model = model.state_dict()  # Save the best model

# After training, load the best model and evaluate on test data
model.load_state_dict(best_model)
test_acc, test_loss = evaluate(test_loader, model, criterion)
print(f"Final Validation Accuracy: {best_val_acc:.1f}%")
print(f"Test Accuracy: {test_acc:.1f}%, Test Loss: {test_loss:.4f}")

Epoch 1, Train Acc: 76.4%, Val Acc: 73.2%
Epoch 2, Train Acc: 77.2%, Val Acc: 74.3%
Epoch 3, Train Acc: 82.8%, Val Acc: 80.2%
Epoch 4, Train Acc: 87.7%, Val Acc: 86.5%
Epoch 5, Train Acc: 90.6%, Val Acc: 89.1%
Epoch 6, Train Acc: 92.1%, Val Acc: 90.9%
Epoch 7, Train Acc: 93.6%, Val Acc: 92.4%
Epoch 8, Train Acc: 93.5%, Val Acc: 92.3%
Epoch 9, Train Acc: 94.0%, Val Acc: 92.8%
Epoch 10, Train Acc: 96.5%, Val Acc: 95.1%
Epoch 11, Train Acc: 96.8%, Val Acc: 95.2%
Epoch 12, Train Acc: 97.5%, Val Acc: 96.2%
Epoch 13, Train Acc: 97.8%, Val Acc: 96.3%
Epoch 14, Train Acc: 98.2%, Val Acc: 96.5%
Epoch 15, Train Acc: 98.3%, Val Acc: 96.6%
Epoch 16, Train Acc: 98.3%, Val Acc: 96.4%
Epoch 17, Train Acc: 98.7%, Val Acc: 97.0%
Epoch 18, Train Acc: 98.7%, Val Acc: 97.1%
Epoch 19, Train Acc: 98.9%, Val Acc: 97.0%
Epoch 20, Train Acc: 99.0%, Val Acc: 97.2%
Epoch 21, Train Acc: 98.9%, Val Acc: 97.2%
Epoch 22, Train Acc: 97.5%, Val Acc: 95.4%
Epoch 23, Train Acc: 99.1%, Val Acc: 97.2%
Epoch 24, Train Acc:

In [ ]:
T_data = np.loadtxt("T_L30.txt")
# Assuming T_test is a tensor or array that contains the temperatures for the test set

T_test_np = T_test.numpy() if isinstance(T_test, torch.Tensor) else T_test

# Prepare data structure to store temperature-wise results
temp_dict = {
    temp.item(): {"correct": 0, "total": 0, "output_sum": np.zeros(2)}
    for temp in torch.unique(T_test)
}

# Evaluate the model for each sample in the test set and collate results by temperature
model.eval()
with torch.no_grad():
    for X, y, temp in zip(
        test_loader.dataset.tensors[0], test_loader.dataset.tensors[1], T_test_np
    ):
        output = model(X.unsqueeze(0))  # Add batch dimension
        pred = output.argmax(dim=1)
        correct = (pred == y).item()

        temp_dict[temp]["correct"] += correct
        temp_dict[temp]["total"] += 1
        temp_dict[temp]["output_sum"] += output.softmax(dim=1).squeeze(0).numpy()

# Calculate average accuracy and average output for each temperature
avg_accuracy_by_temp = {
    temp: data["correct"] / data["total"] for temp, data in temp_dict.items()
}
avg_output_by_temp = {
    temp: data["output_sum"] / data["total"] for temp, data in temp_dict.items()
}

# Sort temperatures for plotting
sorted_temps = sorted(avg_accuracy_by_temp.keys())

# Plotting
plt.figure(figsize=(14, 6))

# Average accuracy plot
plt.subplot(1, 2, 1)
plt.plot(
    sorted_temps, [avg_accuracy_by_temp[temp] for temp in sorted_temps], marker="o"
)
plt.xlabel("Temperature (T/J)")
plt.ylabel("Average Accuracy")
plt.title("Average Accuracy by Temperature")

# Average output neuron plot
plt.subplot(1, 2, 2)
for neuron_index in range(2):
    plt.plot(
        sorted_temps,
        [avg_output_by_temp[temp][neuron_index] for temp in sorted_temps],
        marker="o",
        label=f"Neuron {neuron_index}",
    )
plt.xlabel("Temperature (T/J)")
plt.ylabel("Average Neuron Output")
plt.title("Average Output Neuron Value by Temperature")
plt.legend()

plt.tight_layout()
plt.show()

NameError: name 'T_test' is not defined

In [ ]:
# def evaluate(dataloader, model, loss_fn):
#     model.eval()
#     total_loss, total_correct = 0, 0
#     with torch.no_grad():
#         for X, y in dataloader:
#             pred = model(X)
#             total_loss += loss_fn(pred, y).item()
#             total_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     num_samples = len(dataloader.dataset)
#     avg_accuracy = 100 * total_correct / num_samples
#     avg_loss = total_loss / len(dataloader)
#     return avg_accuracy, avg_loss


# # After training and validation loop
# test_acc, test_loss = evaluate(test_loader, model, criterion)
# print(f"Test Accuracy: {test_acc:.1f}%, Test Loss: {test_loss:.4f}")

# print(f"Final Validation Accuracy: {val_acc:.1f}%")
# print(f"Test Accuracy: {test_acc:.1f}%")

# optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)
# criterion = nn.CrossEntropyLoss()

# epochs = 50
# best_val_acc = 0
# best_model = None

# for epoch in range(epochs):
#     train(train_loader, model, criterion, optimizer)
#     train_acc, train_loss = evaluate(train_loader, model, criterion)
#     val_acc, val_loss = evaluate(val_loader, model, criterion)

#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         best_model = model.state_dict()  # Save the best model

#     print(f"Epoch {epoch+1}, Train Acc: {train_acc:.1f}%, Val Acc: {val_acc:.1f}%")

# # After training, load the best model and evaluate on test data
# model.load_state_dict(best_model)
# test_acc, test_loss = evaluate(test_loader, model, criterion)
# print(f"Final Validation Accuracy: {best_val_acc:.1f}%")
# print(f"Test Accuracy: {test_acc:.1f}%, Test Loss: {test_loss:.4f}")

In [ ]:
#             pred = model(X)
#             total_loss += loss_fn(pred, y).item()
#             total_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     num_samples = len(dataloader.dataset)
#     avg_accuracy = 100 * total_correct / num_samples
#     avg_loss = total_loss / len(dataloader)
#     return avg_accuracy, avg_loss


# # After training and validation loop
# test_acc, test_loss = evaluate(test_loader, model, criterion)
# print(f"Test Accuracy: {test_acc:.1f}%, Test Loss: {test_loss:.4f}")

# print(f"Final Validation Accuracy: {val_acc:.1f}%")
# print(f"Test Accuracy: {test_acc:.1f}%")

# optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)
# criterion = nn.CrossEntropyLoss()

# epochs = 50
# best_val_acc = 0
# best_model = None

# for epoch in range(epochs):
#     train(train_loader, model, criterion, optimizer)
#     train_acc, train_loss = evaluate(train_loader, model, criterion)
#     val_acc, val_loss = evaluate(val_loader, model, criterion)

#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         best_model = model.state_dict()  # Save the best model

#     print(f"Epoch {epoch+1}, Train Acc: {train_acc:.1f}%, Val Acc: {val_acc:.1f}%")

# # After training, load the best model and evaluate on test data
# model.load_state_dict(best_model)
# test_acc, test_loss = evaluate(test_loader, model, criterion)
# print(f"Final Validation Accuracy: {best_val_acc:.1f}%")
# print(f"Test Accuracy: {test_acc:.1f}%, Test Loss: {test_loss:.4f}")

In [ ]:
# Old code:

# def plot_metrics(metrics, title="Model Performance"):
#     plt.figure(figsize=(12, 5))
#     plt.subplot(1, 2, 1)
#     for label, data in metrics["accuracy"].items():
#         plt.plot(data, label=label)
#     plt.title("Accuracy vs. Epochs")
#     plt.xlabel("Epoch")
#     plt.ylabel("Accuracy")
#     plt.legend()

#     plt.subplot(1, 2, 2)
#     for label, data in metrics["loss"].items():
#         plt.plot(data, label=label)
#     plt.title("Loss vs. Epochs")
#     plt.xlabel("Epoch")
#     plt.ylabel("Loss")
#     plt.legend()

#     plt.suptitle(title)
#     plt.show()


# weight_decays = [0, 0.001, 0.01, 0.1]
# metrics = {"accuracy": {}, "loss": {}}

# for decay in weight_decays:
#     model = IsingModelNN()  # Reinitialize model
#     optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=decay)
#     train_accuracies, val_accuracies = [], []
#     train_losses, val_losses = [], []

#     for epoch in range(epochs):
#         train(train_loader, model, criterion, optimizer)
#         train_acc, train_loss = validate(train_loader, model, criterion)
#         val_acc, val_loss = validate(val_loader, model, criterion)
#         train_accuracies.append(train_acc)
#         val_accuracies.append(val_acc)
#         train_losses.append(train_loss)
#         val_losses.append(val_loss)

#     metrics["accuracy"][f"wd={decay}"] = val_accuracies
#     metrics["loss"][f"wd={decay}"] = val_losses

# plot_metrics(metrics, "Impact of L2 Regularization")

In [ ]:
# # weight decay factor added; start with a small value
# weight_decay = 0.01
# optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=weight_decay)
# # "SGD": optim.SGD(model.parameters(), lr=0.01),
# # "SGD with Momentum": optim.SGD(model.parameters(), lr=0.01, momentum=0.9),
# # "Adam": optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)
# criterion = nn.CrossEntropyLoss()


# def train(dataloader, model, loss_fn, optimizer):
#     size = len(dataloader.dataset)
#     model.train()
#     for batch, (X, y) in enumerate(dataloader):
#         pred = model(X)
#         loss = loss_fn(pred, y)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()


# def validate(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     model.eval()
#     loss, correct = 0, 0
#     with torch.no_grad():
#         for X, y in dataloader:
#             pred = model(X)
#             loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     loss /= num_batches
#     correct /= size
#     return 100 * correct, loss


# epochs = 50
# for epoch in range(epochs):
#     train(train_loader, model, criterion, optimizer)
#     train_acc, train_loss = validate(train_loader, model, criterion)
#     val_acc, val_loss = validate(val_loader, model, criterion)
#     print(f"Epoch {epoch+1}, Train Acc: {train_acc:.1f}%, Val Acc: {val_acc:.1f}%")